# Damatura, Nigeria IDP Analysis
##### January 2025

In [1]:
# standard library imports
import os

# third-party imports
import rasterio
import geopandas as gpd

import numpy as np
import pandas as pd

# change to project root directory
os.chdir('../')
print("directory changes")
print(f"current working directory is:", os.getcwd())

# local imports (after changing directory)
from src.population import (
    clip_raster_with_shapefile,
    raster_to_points
)

from src.idp import kmz_to_shapefile


directory changes
current working directory is: c:\Users\Carol\Documents\GitHubCode\damaturu-nigeria-idp


1.0 Population Density

###### 1.1 Check Data CRS

In [2]:
# check coordinate system of population density data for Nigeria, nga_pd__2020_1km_UNadj.tif
with rasterio.open("data/raw/nga_pd_2020_1km_UNadj.tif") as src:
    print(f"coordinate system: {src.crs}")

coordinate system: EPSG:4326


In [8]:
# check coordinate system of aoi, damaturu.shp
aoi = gpd.read_file("data/raw/AOI/damaturu.shp")
print(f"coordinate system: {aoi.crs}")

coordinate system: epsg:4326


###### 1.2 Process Data

In [5]:
# apply "clip_raster_with_shapefile" function to get population density (raster) 
## of Damaturu, Nigeria specifically (shapefile) and sav interim "damaturu_pd" (raster) tif format
clip_raster_with_shapefile(
    "data/raw/nga_pd_2020_1km_UNadj.tif",
    "data/raw/AOI/damaturu.shp",
    "data/interim/damaturu_pd.tif"
)

In [3]:
# gain insight into raster dimensions to best choose a sample rate for raster to point conversion
## (i.e., determine how many points to sample from the raster, 
## where sample_rate = 1 means that every pixel from the raster is converted to a point)
with rasterio.open("data/interim/damaturu_pd.tif") as src:
    print(f"raster dimensions: {src.width} x {src.height}")
    print(f"total pixels: {src.width * src.height}")
    
# note: raster dimensions and total pixels are small, therefore use sample_rate = 1

raster dimensions: 17 x 13
total pixels: 221


In [8]:
with rasterio.open("data/interim/damaturu_pd.tif") as src:
    total_pixels = src.width * src.height
    target_points = 20000  # adjust based on your needs (web viz, aim for 10,000-50,000 points)
    suggested_sample_rate = int(total_pixels / target_points)
    print(f"suggested sample rate: {suggested_sample_rate}")

suggested sample rate: 0


In [4]:
# convert interim (raster) "damaturu_pd" to processed "damaturu_pd_points" points for export and viz
raster_to_points(
    "data/interim/damaturu_pd.tif",
    'data/processed/damaturu_pd_points.geojson',
    sample_rate=1
)


input raster dimensions: 17 x 13
total pixels: 221

Created 221 points
Value range: -99999.00 to 3183.82


,population_density,geometry
0,-99999.0,POINT (11.91125 11.79542)
1,-99999.0,POINT (11.91958 11.79542)
2,-99999.0,POINT (11.92792 11.79542)
3,-99999.0,POINT (11.93625 11.79542)
4,-99999.0,POINT (11.94458 11.79542)
...,...,...
216,-99999.0,POINT (12.01125 11.69542)
217,-99999.0,POINT (12.01958 11.69542)
218,-99999.0,POINT (12.02792 11.69542)
219,-99999.0,POINT (12.03625 11.69542)


2.0 Economic Hotspots

In [3]:
import sys
print(sys.executable)

c:\Users\Carol\Anaconda3\python.exe


In [4]:
#nighttime light data
import ee

3.0 IDP (Location, Location by Site Type, and Change in IDP Locations by Site Types (i.ee, "IDP Growth"))

###### 3.1 Check Data CRS

In [7]:
# convert IDP location KMZ to shapefile
idp_shapefile = kmz_to_shapefile(
    "data/raw/Round 39 North East IDP Camp Sites KMZ.kmz",
    "data/processed/idp_points.shp"
)

AttributeError: 'NoneType' object has no attribute 'Open'